In [3]:
# Team: Rishab Nayak and Jason Li
# Report Link: http://cs-people.bu.edu/rishab/
import cv2
import sys
import numpy as np

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Cannot open the video cam")
    sys.exit()

ret, frame0 = cap.read()
if not ret:
    print("Cannot read a frame from video stream")

while(1):
    ret, frame = cap.read()

    if not ret:
        print("Cannot read a frame from video stream")
        break
    
    hsv_min = np.array([0, 60, 30], dtype = "uint8")
    hsv_max = np.array([30, 255, 255], dtype = "uint8")
    hsv_src = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv_skin_region = cv2.inRange(hsv_src, hsv_min, hsv_max)
    
    blurred = cv2.blur(hsv_skin_region,(10, 10))
    _, thresholded = cv2.threshold(blurred, 200, 255, cv2.THRESH_BINARY)
    
    contours, hierarchy = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_output = np.zeros(np.shape(frame), dtype='uint8')
    
    if len(contours) != 0:
        contour = max(contours, key=lambda x: cv2.contourArea(x))
        boundrec = cv2.boundingRect(contour)
        hull = cv2.convexHull(contour)
        hullnopoints = cv2.convexHull(contour, returnPoints=False)
        defects = cv2.convexityDefects(contour, hullnopoints)
        
        cv2.drawContours(contour_output, contour, -1, (0,0,255), 2)
        cv2.drawContours(contour_output, [hull], -1, (0, 255, 255), 2)
        cv2.rectangle(contour_output, boundrec, (0,255,0))
        
        if defects is not None:
            count = 0
            for i in range(defects.shape[0]):
                s, e, f, d = defects[i][0]
                start = tuple(contour[s][0])
                end = tuple(contour[e][0])
                far = tuple(contour[f][0])
                a = np.sqrt(np.square((end[0] - start[0])) + np.square((end[1] - start[1])))
                b = np.sqrt(np.square((far[0] - start[0])) + np.square((far[1] - start[1])))
                c = np.sqrt(np.square((end[0] - far[0])) + np.square((end[1] - far[1])))
                angle = np.arccos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))
                if angle <= np.pi / 2:
                    count += 1
                    cv2.circle(contour_output, far, 4, [0, 0, 255], -1)
            if count > 0:
                count = count+1
            cv2.putText(contour_output, str(count), (0, 50), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), 2, cv2.LINE_AA)
    
    cv2.imshow('Output', contour_output)

    if cv2.waitKey(30) == 27:
        print("esc key is pressed by user")
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

esc key is pressed by user


-1